In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [ ]:
import json
import os
import datasets
from pathlib import Path
import shutil
import pandas as pd
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#remove the poems that are not in the multim file
# Load JSON data
with open('/content/drive/MyDrive/LLM Project/multim_poem.json', 'r') as file:
  data = json.load(file)

In [ ]:
new_data = []
for item in data:
  path = '/content/drive/MyDrive/LLM Project/images/' + str(item['id']) + '.jpg'
  if os.path.exists(path):
    new_data.append(item)

In [ ]:
print(len(new_data))

7582


In [ ]:
file_path = "/content/drive/MyDrive/LLM Project/multim_mod.json"

# Write the data to the JSON file
with open(file_path, "w") as json_file:
    json.dump(new_data, json_file)

In [ ]:
with open('/content/drive/MyDrive/LLM Project/multim_mod.json', 'r') as file:
  dat = json.load(file)


In [ ]:
print(len(dat))

7582


In [ ]:
print(dat[0])

{'poem': 'what is lovely never dies\nbut passes into other loveliness\nstar-dust or sea-foam flower or winged air', 'image_url': 'https://farm2.staticflickr.com/1086/1002051357_0e9162423e.jpg', 'id': 0}


In [ ]:
files = os.listdir('/content/drive/MyDrive/LLM Project/images')
print(len(files))

7582


In [ ]:
files = os.listdir('/content/drive/MyDrive/LLM Project/Formatted_Data/val')
print(len(files))

758


In [ ]:
def structure_data(source, dest):
    """
    makes the file structure according to huggingface my_dataset_repository

    Args:

    source: folder where all the images are located
    dest: destination of formatted data - will contain 3 folders (train, val , test)
    """
    #get paths for each folder
    train_folder = os.path.join(dest, 'train')
    test_folder = os.path.join(dest, 'test')
    val_folder = os.path.join(dest, 'val')

    #create train,test,val folders
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok = True)
    os.makedirs(val_folder, exist_ok=True)

    # get images in source
    all_images = [i for i in os.listdir(source) if i.endswith('.jpg')]

    # get num of images in each split
    num_train = int(len(all_images)* 0.80)
    num_val =  int(len(all_images)* .10)

    # randomize images
    random.shuffle(all_images)

    # copying images to train folder
    for i in all_images[:num_train]:
        source_path = os.path.join(source, i)
        dest_path = os.path.join(train_folder, i)
        shutil.copy(source_path, dest_path)

    #copy images to val folder
    for i in all_images[num_train:num_train + num_val]:
        source_path = os.path.join(source, i)
        dest_path = os.path.join(val_folder, i)
        shutil.copy(source_path, dest_path)

    # copying images to test folder
    for i in all_images[num_train+num_val:]:
        source_path = os.path.join(source, i)
        dest_path = os.path.join(test_folder, i)
        shutil.copy(source_path, dest_path)

In [ ]:
source_folder = r"/content/drive/MyDrive/LLM Project/images"
dest_folder = r"/content/drive/MyDrive/LLM Project/Formatted_Data"
structure_data(source_folder, dest_folder)

In [ ]:
files = os.listdir('/content/drive/MyDrive/LLM Project/Formatted_Data/train')
print(len(files))
files = os.listdir('/content/drive/MyDrive/LLM Project/Formatted_Data/test')
print(len(files))
files = os.listdir('/content/drive/MyDrive/LLM Project/Formatted_Data/val')
print(len(files))

6065
759
758


In [ ]:
def create_metadata(source_folder, dest_folders):
    """
    Creates metadata.jsonl file for huggingface eg. [{ "file_name" : "image.name" , "text" : "description of the text"} , ...

    Args:
    source_folder: where the text data is currently
    dest_folders: list of train,val and test folders
    """
    with open(source_folder , 'r') as f:
        data = json.load(f)

    poem_captions = []
    for item in data:
      path = str(item['id']) + '.jpg'
      poem_captions.append({"file_name" : path , "text": item['poem']})

    for dest_folder_path in dest_folders:
        os.makedirs(dest_folder_path, exist_ok=True)
        with open(os.path.join(dest_folder_path, 'metadata.jsonl'), 'w') as f:
            for item in poem_captions:
                f.write(json.dumps(item) + '\n')
    return poem_captions

In [ ]:
source_folder = r"/content/drive/MyDrive/LLM Project/multim_mod.json"
dest_folders = [r"/content/drive/MyDrive/LLM Project/Formatted_Data/train",r"/content/drive/MyDrive/LLM Project/Formatted_Data/test", r"/content/drive/MyDrive/LLM Project/Formatted_Data/val"]
dat = create_metadata(source_folder, dest_folders)

In [ ]:
print(os.path.exists("/content/drive/MyDrive/LLM Project/Formatted_Data/train/metadata.jsonl"))
print(os.path.exists("/content/drive/MyDrive/LLM Project/Formatted_Data/test/metadata.jsonl"))
print(os.path.exists("/content/drive/MyDrive/LLM Project/Formatted_Data/val/metadata.jsonl"))

True
True
True


In [ ]:
files = os.listdir('/content/drive/MyDrive/LLM Project/Formatted_Data/train')
print(len(files))
files = os.listdir('/content/drive/MyDrive/LLM Project/Formatted_Data/test')
print(len(files))
files = os.listdir('/content/drive/MyDrive/LLM Project/Formatted_Data/val')
print(len(files))

6066
760
759
